## basic config

In [1]:
import torch
import os
torch.cuda.device_count()

/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


0

In [2]:

def align_faces_2(root_path):
    import os 
    current_path = os.getcwd()
    temp_path = "/home/bean/stylegan3-editing"
    os.chdir(temp_path)
    command =f"""python prepare_data/preparing_faces_parallel.py \
                --mode align \
                --root_path  {root_path} 
                """
    import subprocess
    subprocess.run(command, shell=True)
    os.chdir(current_path)

In [3]:
# DRAGVIDEO_ROOT_PATH = "/Ext_4T_SSD/ASHOK/"
DRAGVIDEO_ROOT_PATH = "/home/bean/"

# keep matching encoder,SG as pairt 
old = {"encoder":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/e4e_ffhq_encode.pt",
       "SG": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/pretrained_models/ffhq.pkl",
       "model_name":"stylegan2",
       "resolution":1024,
       }

new = {"encoder": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/restyle_e4e_ffhq.pt",
         "SG":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/stylegan3_3rdtime_weights/stylegan3-r-ffhq-1024_module.pkl",
         "model_name":"stylegan3",
               "resolution":1024,
         }

new_256 = {"encoder": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/model_weights/restyle_e4e_ffhq.pt",
         "SG":  f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/Stylegan3_1_files/stylegan3-r-ffhqu-256x256.pkl",
         "model_name":"stylegan3",
         "resolution":256,
         }

videos = {
       "obama": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/obama.mp4",
       "man_speaking": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/person_speaking_original.mp4",
       "rahul": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/rahul.mp4",
       "alien_girl": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/alien_girl.mp4",
       "mirrAR": f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/mirrAR.mp4",
       "vsauce_frnd":f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/OLD/original_videos/micheal_friendzone.mp4",
       
}

In [4]:
EXPERIMENT_NAME_POSTFIX = "vsauce_ffmpeg"#"aligned_sg3_&_new_e4e_reconstruction_invstep_200_pti_step_450"
models_set = old
Testing =False
# --------------------------------  


import datetime
lazy_config = {
    "EXP_NAME": str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+"_"+ EXPERIMENT_NAME_POSTFIX,
    "e4e": models_set['encoder'],
    "stylegan2_ada_ffhq": models_set['SG'],
    "video_path":  videos["vsauce_frnd"],
    "model_name" : models_set['model_name'],
    "n_frames" : 150,#120,#200,
    "IMAGE_SIZE": models_set["resolution"],
    "N_STEPS_in_draggan":  "150",
    "editing_function_name":"smile",#"large_eyes",#"large_eyes" # "make_jaw_wider" # "mouth_wide"
    "fps": 24,
    
}

env_config = {
    "DragGAN_dir": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN",
    "Experiment_base_path":f"{DRAGVIDEO_ROOT_PATH}DragVideo/Data_store/experiments/" ,
    "init_exp_dir_shell_path": f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/init_datadirs.sh",
    "dummy_config_path" : f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/configs/dummy",
    
    
}

#hyper parameters in PTI

hyper_config = {
    "max_pti_steps": 350,
    "first_inv_steps":450,
    "max_images_to_invert": 200,
}

if Testing: 
    hyper_config["max_pti_steps"] = 1
    hyper_config["first_inv_steps"] = 350
    hyper_config["max_images_to_invert"] = 5
    lazy_config["N_STEPS_in_draggan"] = '1'
    


# ----------------------------------------------------------------------------------------------


# create experiment data folder structure
Experiment_name = lazy_config["EXP_NAME"]
Experiment_base_path = env_config["Experiment_base_path"]
Experiment_path = os.path.join(Experiment_base_path, Experiment_name)

# change path configs , hyperparameters 
paths_config_dict = {
    #pretrained models
    "e4e": lazy_config["e4e"],
    "stylegan2_ada_ffhq": lazy_config["stylegan2_ada_ffhq"],
    
    # to store tuned stylegan weights
    "checkpoints_dir": os.path.join(Experiment_path,'tuned_SG'),
    # to store latents
    "embedding_base_dir": os.path.join(Experiment_path,'latents'),
    # aligned / processed images
    "input_data_path": os.path.join(Experiment_path,'aligned'),
     "quad_values_path": os.path.join(Experiment_path,'quad_values'),
}



In [5]:
from run_utils import *

init_experiment_dir(Experiment_name,Experiment_base_path,shell_script_path=env_config["init_exp_dir_shell_path"])

# dummy paths_config overwrites the paths_config.py
# dummy_config_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_PTI/configs/dummy'
add_dummy_config_from_dict("hyperparameters.py", hyper_config,ROOT_PATH=env_config["dummy_config_path"])

add_dummy_config_from_dict("paths_config.py", paths_config_dict,ROOT_PATH=env_config["dummy_config_path"])



# add all these configs to log.txt
# --------------------------------  
with open(os.path.join(Experiment_path,'log.txt'), 'a') as f:
    import json
    f.write(f"lazy_config: {json.dumps(lazy_config, indent=4)}\n")
    f.write(f"env_config: {json.dumps(env_config, indent=4)}\n")
    f.write(f"hyper_config: {json.dumps(hyper_config, indent=4)}\n")
    
    



In [6]:
# extract frames from video
#==============================================================================

from importlib.machinery import SourceFileLoader
# imports the module from the given path
# video_utils = SourceFileLoader("video_utils","../utils_draggan/video_utils.py").load_module()

ffmpeg_utils = SourceFileLoader("video_utils","../utils_draggan/ffmpeg_utils.py").load_module()
raw_path = os.path.join(Experiment_path, "raw")

# # video_utils.extract_frames(lazy_config['video_path'], raw_path,n_frames=lazy_config['n_frames'])
ffmpeg_utils.ffmpeg.extract_frames(video_path = lazy_config['video_path'],
                                      output_frames_dir = raw_path,
                                      fps = lazy_config['fps'],
                                      first_n_frames = lazy_config['n_frames'],
)

ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 4.8.2 (GCC) 20140120 (Red Hat 4.8.2-15)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1539667330082/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --disable-doc --disable-openssl --enable-shared --enable-static --extra-cflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-cxxflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-libs='-lpthread -lm -lz' --enable-zlib --enable-pic --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libopenh264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilte

frame=  150 fps=125 q=-0.0 Lsize=N/A time=00:00:06.25 bitrate=N/A speed=5.22x    
video:59472kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown


# start

In [7]:
# # # delete this cell 
# # "/home/bean/DragVideo/Data_store/man_speaking/aligned": input_data_path 
# # "/home/bean/DragVideo/Data_store/man_speaking/quad_values", quad_pkl_path

# raw_path = "/home/bean/DragVideo/Data_store/man_speaking/raw"
# pre_process_images(raw_path, IMAGE_SIZE=1024) # o/p: config.input_data_path ,quad pickle dir


In [8]:
from utils.align_data import pre_process_images
pre_process_images(raw_path, IMAGE_SIZE=lazy_config['IMAGE_SIZE']) # o/p: config.input_data_path ,quad pickle dir
#
# align_faces_2(os.path.join(Experiment_path,'raw'))



 85%|████████▍ | 127/150 [01:41<00:18,  1.27it/s]

local variable 'shape' referenced before assignment


 85%|████████▌ | 128/150 [01:41<00:15,  1.39it/s]

local variable 'shape' referenced before assignment


 86%|████████▌ | 129/150 [01:42<00:14,  1.48it/s]

...

local variable 'shape' referenced before assignment


 99%|█████████▉| 149/150 [01:53<00:00,  1.80it/s]

local variable 'shape' referenced before assignment


100%|██████████| 150/150 [01:54<00:00,  1.31it/s]


saving aligned images...
saving quad values...
done!


In [9]:
from utils.align_data import pre_process_images
from scripts.run_pti import run_PTI

from run_utils_2 import load_generators,export_updated_pickle
from configs import paths_config

use_multi_id_training = True
model_id = run_PTI(use_wandb=False, use_multi_id_training=use_multi_id_training)

generator_type =paths_config.multi_id_model_type if use_multi_id_training else "__"
old_G, new_G = load_generators(model_id, generator_type)
sg_tuned_pkl = export_updated_pickle(new_G,model_id,name = lazy_config["model_name"])

# print(sg_tuned_pkl) # 'QBUXQCXZGWET'



#get landmarks
landmark_path = f"{DRAGVIDEO_ROOT_PATH}DragVideo/DragGAN/_facial-landmarks-recognition"
os.chdir(landmark_path)
print("pwd",os.getcwd())
import sys
sys.path.append(landmark_path)

from main import landmarks, dict_landmarks,show_landmarks,get_landmarks_dir

# to store landmarks
landmarks_dir =  os.path.join(Experiment_path,'landmarks')
processed_images_dir =  os.path.join(Experiment_path,'aligned')

# generate landmarks for all images in processed_images_dir
get_landmarks_dir(processed_images_dir,landmarks_dir)


/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth
current_directory:  /home/bean/DragVideo/DragGAN/_PTI
current_directory:  /home/bean/DragVideo/DragGAN/_PTI
torch.Size([3, 1024, 1024]) 3 1024 1024
Setting up PyTorch plugin "bias_act_plugin"... Done.


  0%|          | 0/450 [00:00<?, ?it/s]

...

torch.Size([3, 1024, 1024]) 3 1024 1024


100%|██████████| 450/450 [00:33<00:00, 13.27it/s]


torch.Size([3, 1024, 1024]) 3 1024 1024


100%|██████████| 350/350 [58:30<00:00, 10.03s/it]


Exporting large updated pickle based off new generator and ffhq.pkl
pwd /home/bean/DragVideo/DragGAN/_facial-landmarks-recognition
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-09-01_01-35-09_vsauce_ffmpeg/landmarks/000.pkl
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-09-01_01-35-09_vsauce_ffmpeg/landmarks/001.pkl
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-09-01_01-35-09_vsauce_ffmpeg/landmarks/002.pkl
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-09-01_01-35-09_vsauce_ffmpeg/landmarks/003.pkl
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-09-01_01-35-09_vsauce_ffmpeg/landmarks/004.pkl
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-09-01_01-35-09_vsauce_ffmpeg/landmarks/005.pkl
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-09-01_01-35-09_vsauce_ffmpeg/landmarks/006.pkl
landmark_path:  /home/bean/DragVideo/Data_store/experiments/2023-09-01_01-35-09_vsauc

# Runs draggan 
all the the below cells use only these config values

In [10]:
# util function to copy an experiment dir
def copy_exp_dir(exp_src):
    import shutil
    exp_dst = exp_src+"_copy"
    shutil.copytree(exp_src, exp_dst)
    return exp_dst
    
# copy_exp_dir("/home/bean/DragVideo/Data_store/experiments/_SAVE_vsauce_ffmpeg_untuned_sg")

In [11]:
# #Run first config cells, then run this
# if True:
#     # just to run draggan seperately
#     import os 
#     a = "/home/bean/DragVideo/Data_store/experiments/sept_1"
#     b = "/home/bean/DragVideo/Data_store/experiments/sept_1/tuned_SG/stylegan3_NXHOAZADQKTA.pkl"
    
#     Experiment_path = a# f"/home/bean/DragVideo/Data_store/experiments/{lazy_config['EXP_NAME']}"	
#     sg_tuned_pkl = b #new["SG"]


#     lazy_config["N_STEPS_in_draggan"]="150"
#     lazy_config["editing_function_name"]= "smile"#"make_jaw_wider"#smile"
#     lazy_config["IMAGE_SIZE"]="1024"

# # /home/bean/DragVideo/Data_store/experiments/sept_1/latents/0.pt
# # /home/bean/DragVideo/Data_store/experiments/_SAVE_vsauce_frnd_120_frames_wide_nose_with_border/latents/barcelona/PTI/000/0.pt

In [12]:
lazy_config["N_STEPS_in_draggan"]="150"


In [13]:
# clean all data from gpu
import torch
torch.cuda.empty_cache()

os.chdir(env_config["DragGAN_dir"])
import subprocess

print(f"sg_tuned_pkl: {sg_tuned_pkl}")

subprocess.call(['python', '_run_dragvideo.py',
                 '--Experiment_path', Experiment_path,
                 '--N_STEPS',       lazy_config["N_STEPS_in_draggan"],
                 '--CHECKPOINT_PATH', sg_tuned_pkl,
                 "--MAX_SIZE",      str(lazy_config["IMAGE_SIZE"]),
                 "--editing_function_name",lazy_config["editing_function_name"],
])
                #  "--verbose",       "False"])


sg_tuned_pkl: /home/bean/DragVideo/Data_store/experiments/2023-09-01_01-35-09_vsauce_ffmpeg/tuned_SG/stylegan2_FGPUAOCZJFFK.pkl


editing_function_name: smile
Total 130 images to be processed
intiating global state....
calling init_images......
Loading "/home/bean/DragVideo/Data_store/experiments/2023-09-01_01-35-09_vsauce_ffmpeg/tuned_SG/stylegan2_FGPUAOCZJFFK.pkl"... Done.
()
{'z_dim': 512, 'c_dim': 0, 'w_dim': 512, 'img_resolution': 1024, 'img_channels': 3, 'mapping_kwargs': {'num_layers': 8, 'embed_features': None, 'layer_features': None, 'activation': 'lrelu', 'lr_multiplier': 0.01, 'w_avg_beta': 0.995}, 'synthesis_kwargs': {'channel_base': 32768, 'channel_max': 512, 'num_fp16_res': 4, 'conv_clamp': 256, 'architecture': 'skip', 'resample_filter': [1, 3, 3, 1], 'use_noise': True, 'activation': 'lrelu'}}
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


  0%|          | 0/150 [00:00<?, ?it/s]/home/bean/.conda/envs/stylegan3/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343964576/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 150/150 [00:34<00:00,  4.33it/s]


intiating global state....
calling init_images......
Loading "/home/bean/DragVideo/Data_store/experiments/2023-09-01_01-35-09_vsauce_ffmpeg/tuned_SG/stylegan2_FGPUAOCZJFFK.pkl"... Done.
()
{'z_dim': 512, 'c_dim': 0, 'w_dim': 512, 'img_resolution': 1024, 'img_channels': 3, 'mapping_kwargs': {'num_layers': 8, 'embed_features': None, 'layer_features': None, 'activation': 'lrelu', 'lr_multiplier': 0.01, 'w_avg_beta': 0.995}, 'synthesis_kwargs': {'channel_base': 32768, 'channel_max': 512, 'num_fp16_res': 4, 'conv_clamp': 256, 'architecture': 'skip', 'resample_filter': [1, 3, 3, 1], 'use_noise': True, 'activation': 'lrelu'}}


100%|██████████| 150/150 [00:34<00:00,  4.37it/s]


...

100%|██████████| 150/150 [00:09<00:00, 16.23it/s]


intiating global state....
calling init_images......
Loading "/home/bean/DragVideo/Data_store/experiments/2023-09-01_01-35-09_vsauce_ffmpeg/tuned_SG/stylegan2_FGPUAOCZJFFK.pkl"... Done.
()
{'z_dim': 512, 'c_dim': 0, 'w_dim': 512, 'img_resolution': 1024, 'img_channels': 3, 'mapping_kwargs': {'num_layers': 8, 'embed_features': None, 'layer_features': None, 'activation': 'lrelu', 'lr_multiplier': 0.01, 'w_avg_beta': 0.995}, 'synthesis_kwargs': {'channel_base': 32768, 'channel_max': 512, 'num_fp16_res': 4, 'conv_clamp': 256, 'architecture': 'skip', 'resample_filter': [1, 3, 3, 1], 'use_noise': True, 'activation': 'lrelu'}}


100%|██████████| 150/150 [00:09<00:00, 16.39it/s]


Done!


0

In [14]:
# empty catched data
import torch
torch.cuda.empty_cache()

# post processing

In [15]:
import os
from tqdm import tqdm
os.getcwd()
# import importlib
# importlib.reload(ffmpeg_utils)

'/home/bean/DragVideo/DragGAN'

In [16]:

from utils import de_alignment 
put_back_the_edited_image = de_alignment.put_back_the_edited_image


In [17]:
def paste_edited_faces_back(dir_path,edited_dir='after_drag',save_dir='after_drag_pasted'):
    raw_dir = os.path.join(dir_path,'raw')
    edited_dir = os.path.join(dir_path,edited_dir)
    quad_dir = os.path.join(dir_path,'quad_values')
    save_dir = os.path.join(dir_path,save_dir)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # get names from edited_dir
    names = [name.split('.')[0] for name in os.listdir(edited_dir)]
    for name in tqdm(names):
        raw_image = os.path.join(raw_dir,name+'.png')
        edited_image = os.path.join(edited_dir,name+'.png')
        quad_path = os.path.join(quad_dir,name+'.pkl')
        save_path = os.path.join(save_dir,name+'.png')
        put_back_the_edited_image(raw_image,edited_image,quad_path,save_path=save_path)


In [18]:
paste_edited_faces_back(Experiment_path,edited_dir="before_drag",save_dir="before_drag_pasted")
paste_edited_faces_back(Experiment_path,edited_dir="after_drag",save_dir="after_drag_pasted")

100%|██████████| 130/130 [00:25<00:00,  5.17it/s]


In [19]:
# =============================================================================
# using ffmpeg to make video
# =============================================================================
before_drag_pasted_dir = os.path.join(Experiment_path,'before_drag_pasted')
after_drag_pasted_dir = os.path.join(Experiment_path,'after_drag_pasted')
videos_dir = os.path.join(Experiment_path,'videos')

ffmpeg_utils.ffmpeg.make_video(before_drag_pasted_dir,
                               video_name="pre_drag_full",
                               video_dir=videos_dir,
                               fps=lazy_config['fps'],
                                 )
ffmpeg_utils.ffmpeg.make_video(after_drag_pasted_dir,
                                 video_name="post_drag_full",
                                    video_dir=videos_dir,
                                    fps=lazy_config['fps'],
                                    )


ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 4.8.2 (GCC) 20140120 (Red Hat 4.8.2-15)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1539667330082/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --disable-doc --disable-openssl --enable-shared --enable-static --extra-cflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-cxxflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-libs='-lpthread -lm -lz' --enable-zlib --enable-pic --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libopenh264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilte

In [20]:
# =============================================================================
# hstack videos
# =============================================================================

video1= os.path.join(videos_dir,"pre_drag_full.mp4")
video2= os.path.join(videos_dir,"post_drag_full.mp4")

# pre_post_drag_full
ffmpeg_utils.ffmpeg.hstack_videos(video1,
                                  video2,
                                  output_dir=videos_dir,
                                  output_name="hstack_pre_post_drag_full",
)

# original_post_drag_full
ffmpeg_utils.ffmpeg.hstack_videos(lazy_config['video_path'],
                                  video2,
                                  output_dir=videos_dir,
                                  output_name="hstack_original_post_drag_full",
)

ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 4.8.2 (GCC) 20140120 (Red Hat 4.8.2-15)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1539667330082/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --disable-doc --disable-openssl --enable-shared --enable-static --extra-cflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-cxxflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-libs='-lpthread -lm -lz' --enable-zlib --enable-pic --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libopenh264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilte

/home/bean/DragVideo/Data_store/experiments/2023-09-01_01-35-09_vsauce_ffmpeg/videos/hstack_pre_post_drag_full.mp4


frame=  130 fps= 93 q=-1.0 Lsize=    1176kB time=00:00:05.29 bitrate=1820.1kbits/s speed=3.77x    
video:1173kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.199741%
[libx264 @ 0x703540] frame I:2     Avg QP:18.46  size: 34414
[libx264 @ 0x703540] frame P:33    Avg QP:20.92  size: 14631
[libx264 @ 0x703540] frame B:95    Avg QP:25.33  size:  6834
[libx264 @ 0x703540] consecutive B-frames:  2.3%  0.0%  2.3% 95.4%
[libx264 @ 0x703540] mb I  I16..4: 15.6% 73.6% 10.9%
[libx264 @ 0x703540] mb P  I16..4:  3.0% 14.1%  0.8%  P16..4: 19.1%  5.4%  1.1%  0.0%  0.0%    skip:56.5%
[libx264 @ 0x703540] mb B  I16..4:  0.3%  0.9%  0.0%  B16..8: 27.1%  3.9%  0.3%  direct: 1.0%  skip:66.5%  L0:49.8% L1:47.3% BI: 2.8%
[libx264 @ 0x703540] 8x8 transform intra:77.4% inter:90.8%
[libx264 @ 0x703540] coded y,uvDC,uvAC intra: 33.3% 31.9% 4.6% inter: 5.5% 7.1% 0.0%
[libx264 @ 0x703540] i16 v,h,dc,p: 42% 28%  7% 23%
[libx264 @ 0x703540] i8 v,h,dc,ddl,ddr,vr,hd,vl,hu: 29% 15% 28%

/home/bean/DragVideo/Data_store/experiments/2023-09-01_01-35-09_vsauce_ffmpeg/videos/hstack_original_post_drag_full.mp4


Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/home/bean/DragVideo/Data_store/OLD/original_videos/micheal_friendzone.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf59.27.100
  Duration: 00:00:13.08, start: 0.000000, bitrate: 1882 kb/s
    Stream #0:0(und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 134 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
    Stream #0:1(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, unknown/bt709/bt709), 1280x720 [SAR 1:1 DAR 16:9], 1740 kb/s, 25 fps, 25 tbr, 12800 tbn, 50 tbc (default)
    Metadata:
      handler_name    : VideoHandler
      encoder         : Lavc59.37.100 libx264
Input #1, mov,mp4,m4a,3gp,3g2,mj2, from '/home/bean/DragVideo/Data_store/experiments/2023-09-01_01-35-09_vsauce_ffmpeg/videos/post_drag_full.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp4